# Activity 1.01: Generating All Possible Sequences of Steps in a Tic-Tac-Toe Game

This activity will explore the combinatorial explosion that is possible when two players play randomly. We will be using a program that, building on the previous results, generates all possible sequences of moves between a computer player and a human player.

Let's assume that the human player may make any possible move. In this example, given that the computer player is playing randomly, we will examine the wins, losses, and draws belonging to two randomly playing players.

Expected output:

```
step 0. Moves: 1
step 1. Moves: 9
step 2. Moves: 72
step 3. Moves: 504
step 4. Moves: 3024
step 5. Moves: 13680
step 6. Moves: 49402
step 7. Moves: 111109
step 8. Moves: 156775
First player wins: 106279
Second player wins: 68644
Draw 91150
Total 266073
```

  > **Hints**  
  >  1. Reuse all the function codes of Steps 1–9 from the Exercise 1.02.
  >  2. Create a function that maps the `all_moves_from_board` function on each element of a list of board spaces/squares. This way, we will have all of the nodes of a decision tree. The decision tree starts with `[ EMPTY_SIGN * 9 ]` and expands after each move.
  >  3. Create a `filter_wins` function that takes finished games out of the list of moves and appends them in an array containing the board states won by the AI player and the opponent player.
  >  4. Create a `count_possibilities` function that prints and returns the number of decision tree leaves that ended with a draw, were won by the first player, and were won by the second player.
  >  5. We have up to nine steps in each state. In the 0th, 2nd, 4th, 6th, and 8th iterations, the AI player moves. In all other iterations, the opponent moves. We create all possible moves in all steps and take out finished games from the move list.
  >  6. Finally, execute the number of possibilities to experience the combinatorial explosion.

In [1]:
from random import choice

winners = [
    [0, 1, 2],
    [0, 3, 6],
    [0, 4, 8],
    [1, 4, 7],
    [2, 4, 6],
    [2, 5, 8],
    [3, 4, 5],
    [6, 7, 8]
]

AI_SIGN = 'X'
EMPTY_SIGN = '-'
OPPONENT_SIGN = 'O'


def print_board(board):
    print("\n{first_row}\n{second_row}\n{third_row}\n".format(first_row=' '.join(board[:3]),
                                                              second_row=' '.join(board[3:6]),
                                                              third_row=' '.join(board[6:])))


def opponent_move(board, row, column):
    index = 3 * (row - 1) + (column - 1)

    if board[index] == EMPTY_SIGN:
        return board[:index] + OPPONENT_SIGN + board[index + 1:]

    return board


def all_moves_from_board(board, sign):
    boards = list()

    for index in range(len(board)):
        if board[index] == EMPTY_SIGN:
            boards.append(board[:index] + sign + board[index + 1:])

    return boards


def ai_move(board):
    return choice(all_moves_from_board(board, AI_SIGN))


def game_won_by(board):
    for winner in winners:
        if board[winner[0]] == board[winner[1]] == board[winner[2]] != EMPTY_SIGN:
            return board[winner[0]]

    return EMPTY_SIGN


def game_loop():
    board = EMPTY_SIGN * 9
    game_ended = False

    while board.count(EMPTY_SIGN) != 0 and not game_ended:
        if board.count(EMPTY_SIGN) % 2 == 0:
            board = ai_move(board)
        else:
            row = int(input('Fila: '))
            col = int(input('Columna: '))
            board = opponent_move(board, row, col)

        print_board(board)
        sign_flag = game_won_by(board)

        if sign_flag != EMPTY_SIGN:
            game_ended = True

            if sign_flag == AI_SIGN:
                print("AI ganadora")
            else:
                print("Jugador ganador")

In [2]:
def map_all_moves_from_board(boards, sign):
    map_boards = list()

    for board in boards:
        map_boards.extend(all_moves_from_board(board, sign))

    return map_boards

In [3]:
def filter_wins(boards, ai_boards, opponent_boards):
    for board in boards:
        if game_won_by(board) == AI_SIGN:
            ai_boards.append(board)
            boards.remove(board)
        elif game_won_by(board) == OPPONENT_SIGN:
            opponent_boards.append(board)
            boards.remove(board)

    return boards, ai_boards, opponent_boards

In [4]:
def count_possibilities():
    boards = [EMPTY_SIGN * 9]
    ai_boards = list()
    opponent_boards = list()

    for i in range(9):
        print(f'Step {i}. Moves: {len(boards)}')

        if i % 2 == 0:
            sign = AI_SIGN
        else:
            sign = OPPONENT_SIGN

        boards = map_all_moves_from_board(boards, sign)
        boards, ai_boards, opponent_boards = filter_wins(boards, ai_boards, opponent_boards)

    print(f'First player wins: {len(ai_boards)}')
    print(f'Second player wins: {len(opponent_boards)}')
    print(f'Draws: {len(boards)}')
    print(f'Total: {len(ai_boards) + len(opponent_boards) + len(boards)}')


In [5]:
count_possibilities()

Step 0. Moves: 1
Step 1. Moves: 9
Step 2. Moves: 72
Step 3. Moves: 504
Step 4. Moves: 3024
Step 5. Moves: 13680
Step 6. Moves: 49402
Step 7. Moves: 111109
Step 8. Moves: 156775
First player wins: 106279
Second player wins: 68644
Draws: 91150
Total: 266073


As you can see, the tree of the board states consists of a total of $266073$ leaves. The `count_possibilities` function essentially implements a BFS algorithm to traverse all the possible states of the game. Notice that we count these states multiple times because placing an $X$ in the top-right corner in Step 1 and placing an $X$ in the top-left corner in Step 3 leads to similar possible states as starting with the top-left corner and then placing an $X$ in the top-right corner. If we implemented the detection of duplicate states, we would have to check fewer nodes. However, at this stage, due to the limited depth of the game, we will omit this step.

A decision tree, however, is identical to the data structure examined by `count_possibilities`. In a decision tree, we explore the utility of each move by investigating all possible future steps up to a certain extent. In our example, we could calculate the utility of the initial moves by observing the number of wins and losses after fixing the first few moves.